In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
from collections import Counter

class Node:
    
    def __init__(self,feature=None,threshold=None,left=None,right=None,*,value=None):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.value=value
        
    def isLeaf(self):
        return self.value is not None
        
class DecisionTree:
    
    def __init__(self,min_sample_split=2,max_depth=100,nFeatures=None):
        
        self.min_sample_split=min_sample_split
        self.max_depth=max_depth
        self.nFeatures=nFeatures
        self.Root=None
        
    
    def fit(self,x,y):
        
        features_len=x.shape[1]
        self.nFeatures=features_len if self.nFeatures == None else min(features_len,self.nFeatures)
        
        self.Root = self._buildTree(x,y)
        
    def _buildTree(self,x,y,depth=0):
        
        nSamples,features=x.shape  # getting latest value of number of samples and features with new split
        nLabels=len(np.unique(y))  # taking length of predicted vals to know if its a pure node
        
        # Stopping Criteria
        # Leaf Node
        if depth > self.max_depth or nSamples < self.min_sample_split or nLabels == 1 :
            leaf_value=self._mostCommonValues(y)
            return Node(value=leaf_value)
        
        
        # Finding Best Split
        feature_idxs=np.random.choice(features,self.nFeatures,replace=False)
        best_threshold , best_feature = self._bestSplit(x,y,feature_idxs)
        
        
        # Child Nodes
        left_child_indexs,right_child_indexs=self._split(x[:,best_feature],best_threshold)
        
        left_Node=self._buildTree(x[left_child_indexs,:],y[left_child_indexs],depth+1)
        right_Node=self._buildTree(x[right_child_indexs,:],y[right_child_indexs],depth+1)
        
        return Node(feature=best_feature,threshold=best_threshold,left=left_Node,right=right_Node)
        
        
    def predict(self,x):
        
        return np.array([self._traverseTree(i,self.Root) for i in x])
    
    
    def _traverseTree(self,x,node):
        
        if node.isLeaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverseTree(x,node.left)
        return self._traverseTree(x,node.right)
            
    
    def _mostCommonValues(self,y):
        counter=Counter(y)
        pred=counter.most_common(1)[0][0]
        return pred
    
    def _gain(self,y,xCol,threshold):
        
        #parent Entropy
        parent_entropy=self._entropy(y)
        
        # Creating Childerens
        left_indexes,right_indexes=self._split(xCol,threshold)
        
        # Checking for Pure Node
        if len(left_indexes) == 0 or len(right_indexes) == 0:
            return 0
        
        # Resulting Entropy
        n=len(y)
        nLeftIndex,nRightIndex=len(left_indexes),len(right_indexes)
        left_entropy,right_entropy=self._entropy(y[left_indexes]),self._entropy(y[right_indexes])
        # Child Entropy Weighted Average 
        
        child_entropy = (nLeftIndex/n) * left_entropy + (nRightIndex/n) * right_entropy
        
        # Information Gain
        Info_Gain = parent_entropy - child_entropy
        
        return Info_Gain
        
        
        
    
    def _split(self,x_col,threshold):
        
        left_indexes=np.argwhere(x_col <= threshold).flatten()
        right_indexes=np.argwhere(x_col > threshold).flatten()
        return left_indexes,right_indexes
    
    
    def _entropy(self,y):
        freq_hist=np.bincount(y)
        p_arr= freq_hist / len(y)
        entropy_val=-np.sum([p*np.log(p) for p in p_arr if p > 0])  # if p < 0 --> entropy ~ 0
        return entropy_val
        
    
    def _bestSplit(self,x,y,feature_idxs):
        best_IG=-1
        split_index,split_threshold=None,None
        
        for index in feature_idxs:
            x_col=x[:,index]
            thresholds=np.unique(x_col)
            
            for thr in thresholds:
                info_gain=self._gain(y,x_col,thr)
                
                if info_gain > best_IG:
                    best_IG = info_gain
                    split_index = index
                    split_threshold =thr
        return split_threshold,split_index
    
def Accuracy(y_true,y_pred):
    return np.sum(y_true == y_pred) / len(y_true) 

In [4]:
breast_cancer=datasets.load_breast_cancer()
X=breast_cancer.data
Y=breast_cancer.target
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=2312,test_size=0.2)

In [5]:
DT=DecisionTree(min_sample_split=2,max_depth=100)
DT.fit(x_train,y_train)

In [6]:
y_pred=DT.predict(x_test)
Score=Accuracy(y_test,y_pred)
print(f"The Accuracy --> {Score*100:.2f} %")

The Accuracy --> 91.23 %
